In [10]:
from __future__ import print_function, division

%matplotlib inline

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np
import pandas as pd

import random

import thinkstats2
import thinkplot
#Scrolled through Ch 12 exercise code and pulled these in as well.
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from IPython.display import display
import statsmodels.tsa.stattools as smtsa

#Pg 145 importing the data and reading it into a pandas dataframe. The parse 5 will take the dates in column 5 and convert them
transactions = pd.read_csv('mj-clean.csv', parse_dates=[5])
transactions.head()


#Pg 146 Dividing the dataset into groups by reported quality and then transforming each group into equally spaced series by computing the mean daily price per gram
def GroupByQualityAndDay(transactions):
    groups = transactions.groupby('quality')
    dailies = {}
    for name, group in groups:
        dailies[name] = GroupByDay(group)
        
    return dailies

#Pg 147 This loop iterates through the groups an dcalls GroupByDay, which computes the daily average price and returns a new DataFrame
def GroupByDay(transactions, func=np.mean):
    grouped = transactions[['date', 'ppg']].groupby('date')
    daily = grouped.aggregate(func)

    daily['date'] = daily.index
    start = daily.date[0]
    one_year = np.timedelta64(1, 'Y')
    daily['years'] = (daily.date - start) / one_year

    return daily #Now have dataframe that has date and ppg as columns, which are grouped by date
dailies = GroupByQualityAndDay(transactions)
#Pg 147 plotting the result from GroupByQualityAndDay into three time series
thinkplot.PrePlot(rows = 3) #making 3 subplots
for i, (name, daily) in enumerate(dailies.items()):
    thinkplot.SubPlot(i+1)
    title = 'price per gram($)' if i==0 else ''
    thinkplot.Config(ylim = [0, 20], title = title)
    thinkplot.Scatter(daily.index, daily.ppg, s=10, label = name)
    #I took out the pyplot part as it kept telling me it wasn't defined even though I've imported it. Also can't figure out why plots label is missing. Looks like it goes high, low, medium though.
    
#Pg 148 creat function that takes the DataFrame of daily prices and computes a least squares fit
def RunLinearModel(daily):
    model = smf.ols('ppg ~ years', data = daily)
    results = model.fit()
    return model, results
#Iterate through the qualities and fit a model to each
for name, daily in dailies.items():
    model, results = RunLinearModel(daily)
    print(name)
    display(results.summary())
#Pg 150 plots the observed prices and the fitted values
def PlotFittedValues(model, results, label=''):
    years = model.exog[:,1]
    values = model.endog
    thinkplot.Scatter(years, values, s=15, label=label)
    thinkplot.Plot(years, results.fittedvalues, label='model', color='#ff7f00')

#Exercise 12-1
#Creating quadratic model to fit the time series of daily prices, and use the model to generate predictions. Used code from chap12soln to help
def RunQuadraticModel(daily):
    daily['years2'] = daily.years**2
    model = smf.ols('ppg ~ years + years2', data=daily)
    results = model.fit()
    return model, results

name = 'high'
daily = dailies[name]

model, results = RunQuadraticModel(daily)
results.summary()    

PlotFittedValues(model, results, label=name)
thinkplot.Config(title='Fitted values',
                 xlabel='Years',
                 xlim=[-0.1, 3.8],
                 ylabel='price per gram ($)')

years = np.linspace(0, 5, 101)
thinkplot.Scatter(daily.years, daily.ppg, alpha=0.1, label=name)
PlotPredictions(daily, years, func=RunQuadraticModel) #For some reason it says that 'PlotPredictions' is not defined. I've gone through the code in the book and from GitHub and can't figure this part out
thinkplot.Config(title='predictions',
                 xlabel='Years',
                 xlim=[years[0]-0.1, years[-1]+0.1],
                 ylabel='Price per gram ($)')

high


C:\Users\Andrea Fox\Anaconda3\envs\ThinkStats2-master\ThinkStats2-master\code\thinkplot.py:182: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  return plt.subplot(rows, cols, plot_number, **options)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ppg   R-squared:                       0.444
Model:                            OLS   Adj. R-squared:                  0.444
Method:                 Least Squares   F-statistic:                     989.7
Date:                Sun, 16 Feb 2020   Prob (F-statistic):          3.69e-160
Time:                        15:43:19   Log-Likelihood:                -1510.1
No. Observations:                1241   AIC:                             3024.
Df Residuals:                    1239   BIC:                             3035.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.4496      0.045    296.080      0.000      13.361      13.539
years         -0.7082      0.023    -31.460      0.000      -0.752      -0.664
==============================================================================
Omnibus:                       56.254   Durbin-Watson:                   1.847
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              128.992
Skew:                           0.252   Prob(JB):                     9.76e-29
Kurtosis:                       4.497   Cond. No.                         4.71
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

low


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ppg   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     35.90
Date:                Sun, 16 Feb 2020   Prob (F-statistic):           2.76e-09
Time:                        15:43:19   Log-Likelihood:                -3091.3
No. Observations:                1179   AIC:                             6187.
Df Residuals:                    1177   BIC:                             6197.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.3616      0.194     27.671      0.000       4.981       5.742
years          0.5683      0.095      5.991      0.000       0.382       0.754
==============================================================================
Omnibus:                      649.338   Durbin-Watson:                   1.820
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6347.614
Skew:                           2.373   Prob(JB):                         0.00
Kurtosis:                      13.329   Cond. No.                         4.85
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

...

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ppg   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     64.92
Date:                Sun, 16 Feb 2020   Prob (F-statistic):           1.82e-15
Time:                        15:43:19   Log-Likelihood:                -2053.9
No. Observations:                1238   AIC:                             4112.
Df Residuals:                    1236   BIC:                             4122.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.8791      0.071    125.043      0.000       8.740       9.018
years          0.2832      0.035      8.057      0.000       0.214       0.352
==============================================================================
Omnibus:                      133.025   Durbin-Watson:                   1.767
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              630.863
Skew:                           0.385   Prob(JB):                    1.02e-137
Kurtosis:                       6.411   Cond. No.                         4.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

NameError: name 'PlotPredictions' is not defined

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000001C2862EBF78> (for post_execute):


ValueError: view limit minimum -0.1 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum -0.1 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

In [11]:
#Exercise 12-2 write a definition for a class named SerialCorrelationTest that extends HypothesisTest. Ue this class to test whether the serial correlation in raw price data is statistically signiifcant. Also test the residuals of the linear model and quadratic model.
#Pg 154 compute serial correlation
def SerialCorr(series, lag = 1):
    xs = series[lag:]
    ys = series.shift(lag)[lag:]
    corr = thinkstats2.Corr(xs, ys)
    return corr
#Creating class named SerialCorrelationTest
class SerialCorrelationTest(thinkstats2.HypothesisTest):

    def TestStatistic(self, data): #Computes the test stastistic
        series, lag = data
        test_stat = abs(SerialCorr(series, lag))
        return test_stat

    def RunModel(self): #Runs the hypothesis
        series, lag = self.data
        permutation = series.reindex(np.random.permutation(series.index))
        return permutation, lag
# test the correlation between consecutive prices    
name = 'high'
daily = dailies[name]

series = daily.ppg
test = SerialCorrelationTest((series, 1))
pvalue = test.PValue()
print(test.actual, pvalue)
# test for serial correlation in residuals of the linear model (from exercise 12-1)
_, results = RunLinearModel(daily)
series = results.resid
test = SerialCorrelationTest((series, 1))
pvalue = test.PValue()
print(test.actual, pvalue)    
# test for serial correlation in residuals of the quadratic model (from exercise 12-1)
_, results = RunQuadraticModel(daily)
series = results.resid
test = SerialCorrelationTest((series, 1))
pvalue = test.PValue()
print(test.actual, pvalue)


0.485229376194738 0.0
0.07570473767506256 0.007
0.056073081612899096 0.051
